In [ ]:
# define stop words
stop_words = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
    "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of",
    "at", "by", "for", "with", "about", "against", "between", "into", "through",
    "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then", "once",
    "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
    "few", "more", "most", "other", "some", "such", "nor", "only",
    "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just",
    "don", "should", "now", "RT"
])

Pre-processing Steps:

1. Convert to lower case
2. Tokenization
3. Remove stop words
4. Lemmatization

In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("ucberkeley-dlab/measuring-hate-speech")

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
# print the text of the first 5 examples before removing stop words
for i in range(5):
    print(df['text'][i])

In [ ]:
# preprocess the data

#make a copy of df for preprocessing
data = df.copy()
# convert to lowercase
data['text'] = data['text'].str.lower()

# convert to list
data['text'] = data['text'].str.split()

# remove stopwords
data['text'] = data['text'].apply(lambda x: [item for item in x if item not in stop_words])

# lemmatize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
data['text'] = data['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
# print the text of the first 5 examples after preprocessing
for i in range(5):
    print(data['text'][i])

In [ ]:
# print the text of the first 5 examples an their classes
for i in range(5):
    print("Example ", i)
    print("Racism:", data['target_race'][i])
    print("Sexism:", data['target_gender'][i])
    print("Xenophobia:", data['target_origin'][i])
    print()

In [ ]:
# find number of rows targeting race, gender, and origin
num_race = 0
num_gender = 0
num_origin = 0
for i, row in data.iterrows():
    if row['target_race'] == 1:
        num_race += 1
    if row['target_gender'] == 1:
        num_gender += 1
    if row['target_origin'] == 1:
        num_origin += 1

print("Num_race:", num_race)
print("Num_gender:", num_gender)
print("Num_origin:", num_origin)


In [ ]:
# go through the data and create 3 lists representing whether each example is racist, sexist, or xenophobic
racism = []
sexism = []
xenophobia = []
for i, row in data.iterrows():
    racism.append(1 if row['target_race'] == 1 else 0)
    sexism.append(1 if row['target_gender'] == 1 else 0)
    xenophobia.append(1 if row['target_origin'] == 1 else 0)

In [ ]:
# Baseline 1: random assignment
# Randomly choose a class and keep track of how accurate the model is
import random
correct_racism = 0
correct_sexism = 0
correct_xenophobia = 0
for i in range(len(racism)):
    if racism[i] == random.randint(0, 1):
        correct_racism += 1
    if sexism[i] == random.randint(0, 1):
        correct_sexism += 1
    if xenophobia[i] == random.randint(0, 1):
        correct_xenophobia += 1

print("Random assignment accuracy for racism:", correct_racism / len(racism))
print("Random assignment accuracy for sexism:", correct_sexism / len(sexism))
print("Random assignment accuracy for xenophobia:", correct_xenophobia / len(xenophobia))


In [ ]:
# Baseline 2: majority class
# Choose the majority class and keep track of how accurate the model is
majority_racism = 1 if num_race > len(racism) / 2 else 0
majority_sexism = 1 if num_gender > len(sexism) / 2 else 0
majority_xenophobia = 1 if num_origin > len(xenophobia) / 2 else 0

correct_racism = 0
correct_sexism = 0
correct_xenophobia = 0
for i in range(len(racism)):
    if racism[i] == majority_racism:
        correct_racism += 1
    if sexism[i] == majority_sexism:
        correct_sexism += 1
    if xenophobia[i] == majority_xenophobia:
        correct_xenophobia += 1

print("Majority class accuracy for racism:", correct_racism / len(racism)) 
print("Majority class accuracy for sexism:", correct_sexism / len(sexism))
print("Majority class accuracy for xenophobia:", correct_xenophobia / len(xenophobia))

In [ ]:
# Create a classifier object which consists of three MLP classifiers, one for each class (racism, sexism, xenophobia)
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import joblib

class Classifier:
    def __init__(self, hidden_layer_sizes=(50,), alpha=0.0005):
        self.racism_classifier = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha)
        self.sexism_classifier = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha)
        self.xenophobia_classifier = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha)

    def fit(self, train_data, train_racism, train_sexism, train_xenophobia):
        self.racism_classifier.fit(train_data, train_racism)
        self.sexism_classifier.fit(train_data, train_sexism)
        self.xenophobia_classifier.fit(train_data, train_xenophobia)

    def predict(self, racism, sexism, xenophobia):
        racism_predictions = self.racism_classifier.predict(racism)
        sexism_predictions = self.sexism_classifier.predict(sexism)
        xenophobia_predictions = self.xenophobia_classifier.predict(xenophobia)

        return racism_predictions, sexism_predictions, xenophobia_predictions
    
    def score(self, racism_pred, racism_true, sexism_pred, sexism_true, xenophobia_pred, xenophobia_true):
        racism_accuracy = f1_score(racism_true, racism_pred)
        sexism_accuracy = f1_score(sexism_true, sexism_pred)
        xenophobia_accuracy = f1_score(xenophobia_true, xenophobia_pred)

        return racism_accuracy, sexism_accuracy, xenophobia_accuracy

In [ ]:
# Prep Training Data

# convert the list of words to strings
data['text'] = data['text'].apply(lambda x: ' '.join(x))

# split the data into training and testing sets
train_data = data['text'][0:len(data) // 10 * 7]
dev_data = data['text'][len(data) // 10 * 7:len(data) // 10 * 8]
test_data = data['text'][len(data) // 10 * 8:]

train_racism = racism[0:len(data) // 10 * 7]
dev_racism = racism[len(data) // 10 * 7:len(data) // 10 * 8]
test_racism = racism[len(data) // 10 * 8:]

train_sexism = sexism[0:len(data) // 10 * 7]
dev_sexism = sexism[len(data) // 10 * 7:len(data) // 10 * 8]
test_sexism = sexism[len(data) // 10 * 8:]

train_xenophobia = xenophobia[0:len(data) // 10 * 7]
dev_xenophobia = xenophobia[len(data) // 10 * 7:len(data) // 10 * 8]
test_xenophobia = xenophobia[len(data) // 10 * 8:]

# create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# fit the vectorizer on the training data
vectorizer.fit(train_data)

# transform the training and testing data
train_data = vectorizer.transform(train_data)
dev_data = vectorizer.transform(dev_data)
test_data = vectorizer.transform(test_data)

In [ ]:
# find best hidden layer node size
classifier = Classifier()
best_accuracy = 0
best_i = 0

for i in range(0, train_data.shape[0] - 1000, 1000):
    classifier.fit(train_data[i:i + 1000], train_racism[i:i + 1000], train_sexism[i:i + 1000], train_xenophobia[i:i + 1000])
    racism_pred, sexism_pred, xenophobia_pred = classifier.predict(dev_data, dev_data, dev_data)
    racism_accuracy, sexism_accuracy, xenophobia_accuracy = classifier.score(racism_pred, dev_racism, sexism_pred, dev_sexism, xenophobia_pred, dev_xenophobia)
    print("Racism accuracy after training on", i + 1000, "examples:", racism_accuracy)
    print("Sexism accuracy after training on", i + 1000, "examples:", sexism_accuracy)
    print("Xenophobia accuracy after training on", i + 1000, "examples:", xenophobia_accuracy)
    print()
    if racism_accuracy + sexism_accuracy + xenophobia_accuracy > best_accuracy:
        best_accuracy = racism_accuracy + sexism_accuracy + xenophobia_accuracy
        best_i = i
        joblib.dump(classifier, "best_classifier.pt")

print("Best average accuracy:", best_accuracy / 3)
print("Found after training on", best_i + 1000, "examples")

In [ ]:
# Predict on the test set
racism_pred, sexism_pred, xenophobia_pred = classifier.predict(test_data, test_data, test_data)
racism_accuracy, sexism_accuracy, xenophobia_accuracy = classifier.score(racism_pred, test_racism, sexism_pred, test_sexism, xenophobia_pred, test_xenophobia)
print("Racism accuracy on test set:", racism_accuracy)
print("Sexism accuracy on test set:", sexism_accuracy)
print("Xenophobia accuracy on test set:", xenophobia_accuracy)